# **Import Module**

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow as tf
import math
import random

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore')

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# **Download Data**

In [93]:
!wget --no-check-certificate https://raw.githubusercontent.com/rioooranteai/data-analytics---modeling/main/Apple%20Stock/Dataset/apple_stock_data.csv -O /content/apple_stock_data.csv

--2025-03-01 04:24:51--  https://raw.githubusercontent.com/rioooranteai/data-analytics---modeling/main/Apple%20Stock/Dataset/apple_stock_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31516 (31K) [text/plain]
Saving to: ‘/content/apple_stock_data.csv’

/content/apple_stoc 100%[===================>]  30.78K  --.-KB/s    in 0.01s   

2025-03-01 04:24:51 (2.91 MB/s) - ‘/content/apple_stock_data.csv’ saved [31516/31516]



# **Read Data**

In [94]:
df = pd.read_csv('/content/apple_stock_data.csv')
df.head()

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-02 00:00:00+00:00,176.665985,177.570007,177.779999,175.460007,175.520004,77334800
1,2023-11-03 00:00:00+00:00,175.750671,176.649994,176.820007,173.350006,174.240005,79763700
2,2023-11-06 00:00:00+00:00,178.317520,179.229996,179.429993,176.210007,176.380005,63841300
3,2023-11-07 00:00:00+00:00,180.894333,181.820007,182.440002,178.970001,179.179993,70530000
4,2023-11-08 00:00:00+00:00,181.958893,182.889999,183.449997,181.589996,182.350006,49340300


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       252 non-null    object 
 1   Adj Close  252 non-null    float64
 2   Close      252 non-null    float64
 3   High       252 non-null    float64
 4   Low        252 non-null    float64
 5   Open       252 non-null    float64
 6   Volume     252 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


In [96]:
df.describe()

,Adj Close,Close,High,Low,Open,Volume
count,252.000000,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,199.088202,199.454286,201.093056,197.608373,199.316032,5.832214e+07
std,21.511933,21.321567,21.571427,20.963034,21.365476,3.025771e+07
min,164.585999,165.000000,166.399994,164.080002,165.350006,2.404830e+07
25%,182.254063,182.852501,184.617504,181.487499,182.777504,4.278855e+07
50%,192.370026,193.084999,194.399994,191.724998,192.989998,5.174865e+07
75%,221.697502,221.697502,224.149994,219.717499,221.635002,6.497460e+07
max,236.479996,236.479996,237.490005,234.449997,236.479996,3.186799e+08


# **Prepare Data**

In [97]:
data = df.copy(deep=True)

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

# **Data Preprocessing**

In [98]:
scaler = MinMaxScaler(feature_range=(0, 1))
data['Close'] = scaler.fit_transform(data[['Close']])

# **Create Sequence: Method 1**

In [109]:
def create_sequences(data, seqlen=60):
    X, y = [],[]
    for i in range(len(data) - seqlen):
        X.append(data[i:i+seqlen])
        y.append(data[i+seqlen])
    return np.array(X), np.array(y)

X, y = create_sequences(data['Close'].values, 30)

# **Train Test Split**

In [110]:
train_size = int(len(data) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# **LSTM Model**

In [111]:
lstm_model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(units=50),
    Dense(1)
])

In [112]:
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.2657
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0312
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0244
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0185
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0120
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0123
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0093
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0086
Epoch 9/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.0083
Epoch 10/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0078


# **Linear Regression Model**

In [113]:
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data = data.dropna()

In [114]:
X_reg = data[['Lag_1', 'Lag_2', 'Lag_3']]
y_reg = data['Close']
X_train_reg, X_test_reg = X_reg[:train_size], X_reg[train_size:]
y_train_reg, y_test_reg = y_reg[:train_size], y_reg[train_size:]

In [115]:
linear_model = LinearRegression()
linear_model.fit(X_train_reg, y_train_reg)

LinearRegression()

# **Prediction**

In [116]:
print("X_test shape:", X_test.shape)


X_test shape: (21, 30)


In [117]:
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
lstm_predictions = lstm_model.predict(X_test_lstm)
lstm_predictions = scaler.inverse_transform(lstm_predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step


In [118]:
lin_predictions = linear_model.predict(X_test_reg)
lin_predictions = scaler.inverse_transform(lin_predictions.reshape(-1, 1))

In [119]:
min_length = min(len(lstm_predictions), len(lin_predictions))
lstm_predictions = lstm_predictions[:min_length]
lin_predictions = lin_predictions[:min_length]

In [120]:
hybrid_predictions = (0.7 * lstm_predictions) + (0.3 * lin_predictions)